<div class="alert alert-block alert-danger" style="color:black">
    <h2> This is an assessed practical</h2>
    The final part of the portfolio is to author the knowledge base for an expert system (chatbot) as described in activity 3. It is worth 100 marks.<br>
    You are given a file containing the set of questions and answers (<code>portfolio-questions-and-responses-2025.txt</code>), together with a jupyter notebook in which to develop your aiml file (<code>AIML_Selfchecker.ipynb</code>), and run it through a copy of the marking and feedback code prior to submission.
</div>

# Workbook 10: Building a bot to respond to customer queries about missing orders

Overview of activities and objectives of this workbook:

1. The first part of this workbook will encode some knowledge about language using some of the AIML tags you learned about in last weeks workbook.
    - We will be creating a chatbot that can act like customer support and respond to customer queries about missing orders.
    - We will use the `<srai>` tag-pairs you learned about last week and also use `<random>` to produce random responses (from a selection) to help test the system.

2. The second part of this workbook will introduce `<that>` tags to embed long-term knowledge and context within your chatbot.
    - `<that>` tags allow your bot respond differently depending on the context of a dialogue.
    - In other words, if the input is the same the bot can change its response depending on what has previously been said.
    - **You will need this to pass the coursework**.

3. The third part of the workbook will introduce wildcards which are able to match to any string in an input `<pattern>` and make categories more general.
    - The `*` and `_` characters can be used as wildcards to match one or more words in a users input `<pattern>` (`_` has a higher priority when matching).
    - The `<star/>` tags are used to match to wildcards in the `<template>`. For example, a word matched in with a `*` in the `<pattern>` can be *referenced* in with the `<star/>` tag in the `<template>`.
    - [This is a useful reference](https://medium.com/pandorabots-blog/wildcards-in-aiml-da7f4a29f42e).

4. The fourth part of the workbook will introduce the `<set>` and `<think>` tags which allow the bot to store variables.
    - The `<set>` tag allows the bot to create a name;value pair (variable) to store information from the input `<pattern>`, i.e. matched with `<star/>`.
    - The `<think>` tag allows the bot to create variables with `<set>` *silently*, i.e. without including it in the response the the user.

5. Finally, we will bring all this together for the coursework.
    - You should use all of these tags to create an AIML file (chatbot) that is able to respond to the 45 questions and answers in the `portfolio-questions-and-responses.txt` file.
    - You get more marks by doing this in as few categories as possible.
    - We provide a `AIML_Selfchecker.ipynb` file to run the marking programme and check your knowledge-base before submission.
</div>

<div style="background:black;width:100%;height:10px"></div><br>

# Part 1: A customer service chatbot

<div class="alert alert-block alert-info" style="color:black"><h2>Activity 1: Encoding knowledge for a customer service chatbot</h2>
Chatbots can store knowledge about the equivalence of all sorts of things, but the most useful is storing knowledge about different ways of saying the same thing.

For example, these are all ways of asking for an order status:
<ul>
    <li>I want to track my order.</li>
    <li> Please give me a status update on my order.</li>
    <li> Where is my order, it hasn't arrived yet.</li>
    <li> Have you sent my order yet.</li>
    <li> Where is my order.</li>
</ul>

<ol>
    <li>Use the skills you learned last week about &lt;srai&gt; tag-pairs to add five new categories to the AIML below.</li>
    <li>Your new categories should encode the knowledge that phrases from the list above are equivalent to the pattern in the category provided.</li>
    <li><b>Run </b> the cell to save your knowledge bade to file.</li>
    <li>The second cell below assumes the customer knows their order number and provides a random response (You'll have to use your knowledge from Web Development to know how it could access a database).</li>
    <li>It uses the &lt;random&gt; tag to randomly pick a response. In this case, randomly pick a scenario where the parcel is delivered, dispatched, out of stock or the order number is not recognised.
    <li>Then <b>run the second cell below</b> to create a test a chatbot.</li>
</ol>
</div>

In [ ]:
%%writefile "data/orders1.aiml"
<aiml>
<category>
  <pattern>TRACK ORDER REQUEST</pattern>
    <template> OK, happy to help. Please enter your order number in the format: my order number is xxxxx</template>
</category>


<!-- add your categories below this line-->

</aiml>


In [ ]:
%%writefile "data/orders2.aiml"
<aiml>
<category>
  <pattern> MY ORDER NUMBER IS *</pattern>
  <template>

     <random>
        <li>Your order has been dispatched and is in transit</li>
        <li>Your order has been marked as delivered.</li>
        <li>Your order number is not recognised.</li>
        <li>We apologise. One or more items in your order were out of stock. We will dispatch as soon as we receive new supplies.</li>
    </random>
 
  </template>
</category>

</aiml>

In [ ]:
# Run this cell to create and chat with your bot
import aiml

# Create the kernel and learn AIML files
myChatbot = aiml.Kernel()
myChatbot.verbose(True)
myChatbot.learn("data/orders1.aiml")
myChatbot.learn("data/orders2.aiml")

print(f"Successfuly loaded {myChatbot.numCategories()} categories")
print(" Is this the number you expected?")

# type bye to end the chat session
keepGoing = True

while keepGoing:
    nextInput = input("Enter your message >> ")
    if nextInput == "bye":
        keepGoing = False
    else:
        print(myChatbot.respond(nextInput))

<div style="background:black;width:100%;height:10px"></div><br>

# Part 2: Embedding historical context using `<that>` tags

In natural language we often say things that only make sense in the context of the previous dialogue - for example the word *it* in the dialogues below.
> What is this module about?  
> Artificial Intelligence  
> Does it have ethical issues?  
> Yes, there are risks of introducing bias against some groups of people.

> What are you eating?  
> An apple  from my garden.  
> Does it have ethical issues?  
> No, it's just a piece of fruit

AIML `<that>` tag pairs let us use refer to the last thing that the bot said - **effectively we add a second condition to the matching process**.
- `<that>THING TO MATCH</that>` goes between the `<pattern>   </pattern>` and the `<template> </template>` parts of the category
- We can also use the `<that/>` tag inside a template to repeat the last thing we the bot said
 - note that like `<star/>`, this is a single tag rather than a tag-pair enclosing contents
   in this case the backslash comes at the end. 
- Note that **the text inside the &lt;that&gt; tags should be NORMALISED (in upper case and without its punctuation)**.

**Read the AIML in the next cell** and make sure you understand what is happening. Then **run** the next two cells to test the 'bad joke' with `<that>` tags.

In [ ]:
%%writefile "data/badJoke.aiml"
<aiml version="1.0.1" encoding="UTF-8">

<category>
  <pattern> TELL ME A JOKE</pattern>
  <template> 
    <random>
      <li> Why did the chicken cross the road?</li>
      <li> What do you get when you cross a sheep with a kangaroo?</li>
      <li> Why don't ants get sick?</li>
    </random>
  </template>
</category>

<category>
  <pattern> I DO NOT KNOW </pattern>
  <that>WHY DID THE CHICKEN CROSS THE ROAD</that>
  <template> To get to the other side</template>
</category>

<category>
  <pattern> I DO NOT KNOW </pattern>
  <that>WHAT DO YOU GET WHEN YOU CROSS A SHEEP WITH A KANGAROO</that>
  <template> A woolly jumper</template>
</category>

<category>
  <pattern> I DO NOT KNOW </pattern>
  <that>WHY DO NOT ANTS GET SICK</that>
  <template> Because they have little anty-bodies</template>
</category>

<category>
  <pattern> I DO NOT KNOW</pattern>
  <template> Default answer. Last thing the bot said was:<that/></template>
</category>
</aiml>

In [ ]:
import aiml

jokeBot = aiml.Kernel()
jokeBot.learn("data/badJoke.aiml")

# type bye to end the chat session
keepGoing = True

while keepGoing:
    nextInput = input("Enter your message >> ")
    if nextInput == "bye":
        keepGoing = False
    else:
        print(jokeBot.respond(nextInput))

<div style="background:black;width:100%;height:10px"></div><br>

# Part 3: Matching inputs with Wildcards `_`, `*` and the `star` tag

The wild card characters `_` and `*` can match one or more words in the input `<pattern>` which allows the `<categories>` to be more flexible/general. For example, the `<pattern>HELLO *</pattern>` would match any input the began with "hello" and had 1 or more words after. A good reference can be found [here](https://medium.com/pandorabots-blog/wildcards-in-aiml-da7f4a29f42e).

The `<star/`> tag can be used to reference words matched with a wildcard in the `<template>`. For example, with this category:

```
<category>
   <pattern>MY NAME IS *</pattern>
   <template>Hi <star/></template>
</category>
```
If the user inputs "My name is Nathan", the bot would respond "Hi Nathan".

You can also use `<star index="x"/>` (where x is an integer) to access words at different indexes of the string matched by a wildcard. For example, with this category:

```
<category>
   <pattern>MY NAME IS *</pattern>
   <template>Hi <star index="1"/></template>
</category>
```
If the user inputs "My name is Nathan Duran", the bot would respond "Hi Nathan" (and ignore the surname).

<div class="alert alert-block alert-info" style="color:black"><h2>Activity 2: Using wildcards and historical context in your order-tracking bot</h2>

These are all ways of asking about a location:
- Where is it?
- Where do I find it?
- I can't find it
- I don't know where to find it.
- Can you tell me where to find it.

<ol>
    <li> Create 5 categories embedding the knowledge that the five phrases above are all equivalent to "GIVE LOCATION".</li>
    <li> Create two new categories, both with the pattern "GIVE LOCATION" but with different &lt;that&gt; tag-pairs. So whether they are matched depends on what the bot last said:
        <ul>
            <li> If the last response was "Your order has been marked as delivered".<br>
                The response should be "It may be in your safe place or with a neighbour"</li>
        <li> If the last response was "OK, happy to help. Please enter your order number in the format: my order number is xxxxx".<br>
                The bot should reply with "Your order number is in your confirmation email".</li>
        </ul>
    <li>Run the code cell enough times to test your bot thoroughly</li>
    <li>Next edit the categories so that where appropriate they use the &lt;star/&gt; tag in the &lt;template&lt; and wildcards in the &lt;pattern&lt; to include the customer's order number in the bot's response.</li>
    <li> Then run your bot again to test it.</li>
</ol>
</div>

In [ ]:
%%writefile "data/orders3.aiml"
<aiml>


<!-- add your five categories using srai in the template to match the phrases from the list  onto GIVE LOCATION -->


<!-- complete these two categories to respond to different conversation history using <that> tag-pairs-->

<category>
 <pattern>GIVE LOCATION</pattern>
 <!-- add that tag-pair here -->
 <template>It may be in your safe place or with a neighbour</template>
</category>

<category>
 <pattern>GIVE LOCATION</pattern>
 <!-- add that tag-pair here -->
 <template>Your order number is in your confirmation email</template>
</category>

</aiml>

In [ ]:
import aiml
myChatbot.learn("data/orders3.aiml")

print(f"Successfuly loaded {myChatbot.numCategories()} categories")
print(" Is this the number you expected?")

# type bye to end the chat session
keepGoing = True

while keepGoing:
    nextInput = input("Enter your message >> ")
    if nextInput == "bye":
        keepGoing = False
    else:
        print(myChatbot.respond(nextInput))

<div style="background:black;width:100%;height:10px"></div><br>

# Part 4: Using `think`, `set` and `get` tags to store and use variables

The `<set>` and `<get>` tags effectively allow the bot to create variables and remember things. These are very useful when used with wildcards and `<star/>`! For example, with this category:

```
<category>
   <pattern>MY NAME IS *</pattern>
   <template>Hi <set name = "username"> <star/></template>
</category>
```
If the user inputs "My name is Nathan", the bot would respond "Hi Nathan". **And also create a variable** called "username" with the value "Nathan" that can be used later. Then, for example with this category:

```
<category>
   <pattern>Good Night</pattern>
   <template>
         Thanks for the conversation <get name = "username"/>!
    </template>
</category>
```
If the user inputs "Good Night", the bot would respond "Thanks for the conversation Nathan!".

The `<think>` tag allows the bot to store variables with `<set>`, but does not output that value to the user. For example, with this category:

```
<category>
   <pattern>MY NAME IS *</pattern>
   <template>
        Hi! <think><set name = "username"> <star/> </think>
    </template>
</category>
```
If the user inputs "My name is Nathan", the bot would respond "Hi!". It would still create a variable called "username" with the value "Nathan" but would not output that.

<div class="alert alert-block alert-info" style="color:black"><h2>Activity 3: Using set and think tag-pairs and conditions to remember order status and respond differently to help requests</h2>

<ol>
    <li>Edit the categories stored in data/orders2.aiml using &lt;think&gt; and &lt;set&gt; tags inside each of the random responses to save an appropriate value describing the order status in a variable called "status"</li>
    <li>Then edit the aiml in the cell below so that it politely directs the user to one of these emails depending on the order status:
        <ul>
            <li>transit.enquiries@acme.com</li>
            <li>missing.delivery@acme.com</li>
            <li>lost.orders@acme.com</li>
            <li>stock.info@acme.com</li>
        </ul>
    <li> Then run the second cell below to run your bot again to test it.</li>
</ol>
</div>

In [ ]:
%%writefile "data/order4.aiml"
<aiml>

<!-- complete this  category using condition tages in the template-->

<category>
 <pattern>I WANT MORE HELP</pattern>
  <template>
    We are sorry that you are not satisfied. 
    Please use the email address below to contact the right team who can give you more help.
       <!-- add add your condition code here -->
   </template>
</category>


</aiml>

In [ ]:
import aiml

myChatbot.learn("data/order4.aiml")
myChatbot.verbose(True)
print(f"Successfuly loaded {myChatbot.numCategories()} categories")
print(" Is this the number you expected?")

# type bye to end the chat session
keepGoing = True

while keepGoing:
    nextInput = input("Enter your message >> ")
    if nextInput == "bye":
        keepGoing = False
    else:
        print(myChatbot.respond(nextInput))

<div style="background:black;width:100%;height:10px"></div><br>

# Part 5: Final portfolio submission - creating a knowledge-base for a chatbot

<div class="alert alert-block alert-info" style="color:black"><h2>Activity 4: A chatbot to answer the portfolio questions</h2>

<h3><b>100 marks</b> - using the marking scheme below</h3>

You should now have knowledge of all of the AIML concepts and tags that you need to complete the final part of the portfolio.

In this weeks folder you will find a file called `portfolio-questions-and-responses-2025.txt` which contains 45 questions and corresponding answers. The task is to author a single file AIML  code that can be loaded into a bot and correctly answer all 45 questions.

You will score more marks for using the knowledge representation language features supported by AIML to reduce the redundancy in  your knowledge base by using as few categories as possible.

We have provided an `AIML_Selfchecker.ipyb` for you to develop and test your chatbot. You can use it to test your AIML using exactly the same code as on the marking server.

1. Open the notebook `AIML_Selfchecker.ipyb` in this directory.
    - The first cell is for you to develop you aiml code in.
        - When you run that cell it writes your aiml to a file called `student.aiml`
        - **Do not change the name of the file produced (i.e. leave it as student.aiml)**

    - Subsequent cells in the notebook run a copy of the same code that is on the marking system.
        - **Note that every time the marking system runs it asks the question in a different order**.
        - But it does keep together the 3 questions that naturally follow on from a particular response.

    - It should give you output on screen and also produce two files:
        - "student-feedback" with a copy of what is shown on screen.
        - "student-responses.txt" which shows you eaach question in turn, what was expected, and what your bot produced, nicely aligned so you can check for typos, missing punctuation etc.

2. Using the ideas from week 9 and 10, edit the first cell (which produces "student.aiml") to contain categories encoding all the information you need to answer the set of 45 questions in `portfolio-questions-and-responses-2025.txt`.
    - We suggest you might start with 45 categories and get your bot correctly answering all the questions.

3. You will need to use some categories with `<that>` tag-pairs to answer three of the questions correctly, because they rely on the previous question to provide context.

4. Then use what you have learned about `<srai>` tags and wildcards to reduce the number of categories in your file - **this will get you more marks**.

5. For even more marks try including `<think>`, `<set>` and `<get>` tags and looking at the lecture notebook to see how `<condition>` tags work in AIML.

6. Once you have completed your AIML you just need to submit the file `student.aiml` produced by the `AIML_Selfchecker.ipyb` notebook.

**Make sure your "student.aiml" file is somewhere that is automatically backed up (like your UWE onedrive).**

</div>

### Marking scheme



| 0-44                                   | 45-90                                  | 100                                            |
|----------------------------------------|----------------------------------------|------------------------------------------------|
| Incorrect responses to some questions. | Correct response to each question.     | Correct response to each question.             | 
|----------------------------------------|----------------------------------------|------------------------------------------------|
|                                        | AIMl language features are used well   |  Exceptional analysis of knowledge to be stored|
|                                        | to provide concise knowledge management|  and how to encode it concisely.               |
|                                        | Solutions that duplicate information   | Wide range of AIMl language features are used. |
|                                        | will get 65 marks.                     |                                                |
|----------------------------------------|----------------------------------------|------------------------------------------------|
| One mark per correct answer.           | 90-N marks,   where  10< N<45          | Fewer than 10 categories used.                 |
|                                        | is the number of AIML categories  used |                                                |
|                                        | to encode the knowledge base.          |                                                |


<div class="alert alert-block alert-success" style="color:black"><b>Save and close Jupyter:</b>
    <ol>
        <li>Use the jupyterlab functions to download your work (ask your tutor if you need help with this) and save it somewhere sensible so you can find it easily.</li>
        <li>Shutdown the notebook when you have finished with this tutorial (menu->file->close and shutdown notebook)</li>
    </ol>
</div